In [1]:
import warnings 
warnings.filterwarnings('ignore')
%autosave 60

Autosaving every 60 seconds


In [2]:
!pip install alpha_vantage

In [3]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange

In [4]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=GBP&to_symbol=USD&apikey=GMLKADN2XG9IP5DG'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)', '2. From Symbol': 'GBP', '3. To Symbol': 'USD', '4. Output Size': 'Compact', '5. Last Refreshed': '2023-12-08 21:55:00', '6. Time Zone': 'UTC'}, 'Time Series FX (Daily)': {'2023-12-08': {'1. open': '1.25868', '2. high': '1.26013', '3. low': '1.25018', '4. close': '1.25483'}, '2023-12-07': {'1. open': '1.25574', '2. high': '1.26126', '3. low': '1.25440', '4. close': '1.25925'}, '2023-12-06': {'1. open': '1.25939', '2. high': '1.26134', '3. low': '1.25512', '4. close': '1.25594'}, '2023-12-05': {'1. open': '1.26316', '2. high': '1.26522', '3. low': '1.25769', '4. close': '1.25923'}, '2023-12-04': {'1. open': '1.26960', '2. high': '1.27245', '3. low': '1.26033', '4. close': '1.26326'}, '2023-12-01': {'1. open': '1.26224', '2. high': '1.27161', '3. low': '1.26136', '4. close': '1.27081'}, '2023-11-30': {'1. open': '1.26948', '2. high': '1.27108', '3. low': '1.26030', '4. close': '1.26217'}, '2023-11-29': {'1. o

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [6]:
df = pd.DataFrame(data['Time Series FX (Daily)']).T
df = df.astype(float)

In [7]:
# Extract features and labels
features = df[['1. open', '2. high', '3. low', '4. close']].values
labels = df['4. close'].shift(-1).fillna(method='ffill').values

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [12]:
# Convert the data to a DataFrame
df = pd.DataFrame(data['Time Series FX (Daily)']).T

# Convert index to datetime
df.index = pd.to_datetime(df.index)

# Convert data types to float
for col in df.columns:
    df[col] = df[col].astype(float)

# Feature Engineering
df['previous_close'] = df['4. close'].shift(1)

# Drop missing values
df.dropna(inplace=True)

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Define sequence length
sequence_length = 5  # Length of the sequences

# Generate sequences for LSTM
sequences = []
targets = []

for i in range(len(df) - sequence_length):
    sequences.append(scaled_data[i:i + sequence_length])
    targets.append(scaled_data[i + sequence_length])

# Convert sequences and targets to numpy arrays
sequences = np.array(sequences)
targets = np.array(targets)

# Split data into training and testing sets
train_size = int(len(sequences) * 0.8)  # 80% for training
test_size = len(sequences) - train_size

train_sequences = sequences[:train_size]
train_targets = targets[:train_size]

test_sequences = sequences[train_size:]
test_targets = targets[train_size:]

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, df.shape[1])))
model.add(LSTM(units=50))
model.add(Dense(units=df.shape[1]))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_sequences, train_targets, epochs=50, batch_size=32, validation_data=(test_sequences, test_targets), callbacks=[early_stopping])

# Predict future values (next 'n_days_to_predict' days)
n_days_to_predict = 5
predicted_values = []

# Take the last 'sequence_length' days as the initial sequence for prediction
current_sequence = sequences[-1]

for _ in range(n_days_to_predict):
    # Reshape the current sequence for prediction
    current_sequence = current_sequence.reshape((1, sequence_length, df.shape[1]))

    # Predict the next day's values (close, high, low)
    next_day_prediction = model.predict(current_sequence)
    predicted_values.append(next_day_prediction[0])

    # Update the current sequence to include the predicted value
    current_sequence = np.append(current_sequence[:, 1:, :], [next_day_prediction], axis=1)

# Inverse transform the predicted values to get actual values
predicted_values = scaler.inverse_transform(predicted_values)

# Display the predicted values for the next 'n_days_to_predict' days
for i, pred in enumerate(predicted_values, 1):
    print(f"Predicted values for Day {i}: Close - {pred[0]}, High - {pred[1]}, Low - {pred[2]}")

    



Epoch 1/50

3/3 [==============================] - 3s 267ms/step - loss: 0.1606 - val_loss: 0.5179
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1293 - val_loss: 0.4085
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0997 - val_loss: 0.2928
Epoch 4/50
3/3 [==============================] - 0s 16ms/step - loss: 0.0677 - val_loss: 0.1752
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0401 - val_loss: 0.0724
Epoch 6/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0195 - val_loss: 0.0157
Epoch 7/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0159 - val_loss: 0.0084
Epoch 8/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0200 - val_loss: 0.0082
Epoch 9/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0175 - val_loss: 0.0140
Epoch 10/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0147 - val_loss: 0.0261
Epoch 11/50
3/3 [========

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Inverse transform the test targets to get actual values
actual_test_targets = scaler.inverse_transform(test_targets)

# Inverse transform the predicted values to get actual values
predicted_values_inverse = scaler.inverse_transform(model.predict(test_sequences))

# Calculate Mean Squared Error (MSE) for each feature
mse_list = []
for i in range(actual_test_targets.shape[1]):
    mse = mean_squared_error(actual_test_targets[:, i], predicted_values_inverse[:, i])
    mse_list.append(mse)
    print(f"MSE for feature {i + 1}: {mse}")

# Calculate Mean Absolute Error (MAE) for each feature
mae_list = []
for i in range(actual_test_targets.shape[1]):
    mae = mean_absolute_error(actual_test_targets[:, i], predicted_values_inverse[:, i])
    mae_list.append(mae)
    print(f"MAE for feature {i + 1}: {mae}")

# Calculate Root Mean Squared Error (RMSE) for each feature
rmse_list = [np.sqrt(mse) for mse in mse_list]
for i, rmse in enumerate(rmse_list, 1):
    print(f"RMSE for feature {i}: {rmse}")


1/1 [==============================] - 0s 16ms/step
MSE for feature 1: 5.9542584476398934e-05
MSE for feature 2: 3.966282420269424e-05
MSE for feature 3: 4.386124537739107e-05
MSE for feature 4: 8.614555709410722e-05
MSE for feature 5: 7.773470739604593e-05
MAE for feature 1: 0.005887106453744976
MAE for feature 2: 0.004843101802625161
MAE for feature 3: 0.005404266140586457
MAE for feature 4: 0.0079521055020784
MAE for feature 5: 0.007673441842731683
RMSE for feature 1: 0.007716384158166241
RMSE for feature 2: 0.006297842821371
RMSE for feature 3: 0.00662278229880698
RMSE for feature 4: 0.009281463090165646
RMSE for feature 5: 0.008816728837615792
